# Persist HTL Data into SQLite

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

## Description

The motive of this notebook is to demonstrate a way of persisting Historical Tonnage List Data into a local Database.  
For this purpose we will use **SQLite3**  


SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine(https://www.sqlite.org/index.html).  
Sqlite3 is a built-in module within Python, so there are no installation and configuration process.   
In order to use it, we only need to import it.

In [3]:
import sqlite3

### Step 1. Create the Database


In [4]:
#code to create a database named HistoricalTonnageList and also connect to it.
conn = sqlite3.connect('HistoricalTonnageList.db')

You will notice that there is a file HistoricalTonnageList.db has been created under the working directory. This is the database file that we have just created.  
This file is created only once, when the above code is executed for the first time. After that the purpose of the code is just to connect to the database.

### Step 2. Execute a query over the HTL api

For the sake of this example we are going to retrieve data for all aframaxes for the last seven days. 

In [5]:
from datetime import date, timedelta,time
from signal_ocean import PortAPI, VesselClassAPI,PortFilter,VesselClassFilter,Connection
from signal_ocean.historical_tonnage_list import HistoricalTonnageListAPI, VesselFilter


connection = Connection(signal_ocean_api_key)

port_api = PortAPI(connection)
vessel_class_api = VesselClassAPI(connection)
htl_api = HistoricalTonnageListAPI(connection)
vessel_class = vessel_class_api.get_vessel_classes(VesselClassFilter(name_like='aframax'))[0]
port = port_api.get_ports(PortFilter(name_like='ceyhan'))[0]
laycanEndInDays = 512
start_date = date.today() - timedelta(days=7)
end_date = date.today()

htl = htl_api.get_historical_tonnage_list(
        port,
        vessel_class,
        laycanEndInDays,
        start_date,
        end_date,
        time=time(hour=6)
    )

htl_df = htl.to_data_frame()
print(htl_df.shape)
htl_df.head()

(8533, 23)


name vessel_class ice_class  \
date                      imo                                               
2021-03-16 06:00:00+00:00 9215050        Alboran I      Aframax       NaN   
                          9038878              Eos      Aframax       NaN   
                          9258894  Golden Splendor      Aframax       NaN   
                          9138604  Barakuda Natuna      Aframax       NaN   
                          9226970          Yoselin      Aframax       NaN   

                                   year_built  deadweight  length_overall  \
date                      imo                                               
2021-03-16 06:00:00+00:00 9215050        2001      109326          244.60   
                          9038878        1993       99440          243.97   
                          9258894        2004      114809          251.60   
                          9138604        1998      109277          244.60   
                          9226970        2001       98893          248.00   

                                   breadth_extreme subclass  \
date                      imo                                 
2021-03-16 06:00:00+00:00 9215050               42    Dirty   
                          9038878               46    Dirty   
                          9258894               44    Dirty   
                          9138604               42    Clean   
                          9226970               43    Dirty   

                                  market_deployment_point_in_time  \
date                      imo                                       
2021-03-16 06:00:00+00:00 9215050                            Spot   
                          9038878                         Program   
                          9258894                            Spot   
                          9138604                        Contract   
                          9226970                        Contract   

                                  push_type_point_in_time  ...  \
date                      imo                              ...   
2021-03-16 06:00:00+00:00 9215050              Not Pushed  ...   
                          9038878              Not Pushed  ...   
                          9258894              Not Pushed  ...   
                          9138604              Not Pushed  ...   
                          9226970              Not Pushed  ...   

                                  commercial_operator_point_in_time  \
date                      imo                                         
2021-03-16 06:00:00+00:00 9215050               Monte Nero Maritime   
                          9038878                             PDVSA   
                          9258894                      Soechi Lines   
                          9138604                      Soechi Lines   
                          9226970              Lundqvist Rederierna   

                                  commercial_status_point_in_time  \
date                      imo                                       
2021-03-16 06:00:00+00:00 9215050                       Available   
                          9038878                       Available   
                          9258894                       Available   
                          9138604                       Available   
                          9226970                       Available   

                                          eta_point_in_time  \
date                      imo                                 
2021-03-16 06:00:00+00:00 9215050 2019-07-15 01:00:00+00:00   
                          9038878 2020-08-04 20:00:00+00:00   
                          9258894 2020-08-05 03:00:00+00:00   
                          9138604 2020-09-12 17:00:00+00:00   
                          9226970 2020-11-19 08:00:00+00:00   

                                   latest_ais_point_in_time  \
date                      imo                                 
2021-03-16 06:00:00+00:00 9

### Step 3. Import the htl_df rows into the Database that we created

In order to import the data we have to install sqlalchemy:
```
pip install sqlalchemy
```

In [6]:
from sqlalchemy import types,create_engine
from os import path
import os

# we create an sqlalchemy engine for the db that we have created
engine = create_engine(f'sqlite:///{path.join(os.path.abspath(os.getcwd()), "HistoricalTonnageList.db")}')

#create a table and append the data
htl_df.to_sql('aframaxes',engine,index=True,if_exists="append")

### Step 4. Verify that the data have been correctly imported

In [7]:
import pandas as pd

ht_from_db_df = pd.read_sql_query("select * from aframaxes", conn)
ht_from_db_df.head()

,date,imo,name,vessel_class,ice_class,year_built,deadweight,length_overall,breadth_extreme,subclass,...,commercial_operator_point_in_time,commercial_status_point_in_time,eta_point_in_time,latest_ais_point_in_time,open_prediction_accuracy_point_in_time,open_country_point_in_time,open_narrow_area_point_in_time,open_wide_area_point_in_time,availability_port_type_point_in_time,availability_date_type_point_in_time
0,2021-03-16 06:00:00.000000,9215050,Alboran I,Aframax,None,2001,109326,244.60,42,Dirty,...,Monte Nero Maritime,Available,2019-07-15 01:00:00.000000,2019-06-13 11:34:48.000000,Port,China,South China,China / Taiwan,Source,Prediction
1,2021-03-16 06:00:00.000000,9038878,Eos,Aframax,None,1993,99440,243.97,46,Dirty,...,PDVSA,Available,2020-08-04 20:00:00.000000,2020-01-05 20:49:03.000000,Narrow Area,"Venezuela, Bolivarian Republic of",Caribs,Caribs,Prediction,Prediction
2,2021-03-16 06:00:00.000000,9258894,Golden Splendor,Aframax,None,2004,114809,251.60,44,Dirty,...,Soechi Lines,Available,2020-08-05 03:00:00.000000,2020-07-16 04:34:32.000000,Narrow Area,Singapore,Singapore / Malaysia,South East Asia,Source,Source
3,2021-03-16 06:00:00.000000,9138604,Barakuda Natuna,Aframax,None,1998,109277,244.60,42,Clean,...,Soechi Lines,Available,2020-09-12 17:00:00.000000,2020-08-23 01:45:14.000000,Narrow Area,Malaysia,Singapore / Malaysia,South East Asia,Source,Source
4,2021-03-16 06:00:00.000000,9226970,Yoselin,Aframax,None,2001,98893,248.00,43,Dirty,...,Lundqvist Rederierna,Available,2020-11-19 08:00:00.000000,2020-10-20 05:42:52.000000,Narrow Area,"Bonaire, Sint Eustatius and Saba",Caribs,Caribs,Prediction,Prediction


### Step 5. Update the created table

By executing the following block of code we can update the aframaxes table that we created in the previous steps.

In [8]:
#connect to db
conn = sqlite3.connect('HistoricalTonnageList.db')
c = conn.cursor()
c.execute('select max(date(date)) from aframaxes')

#get the most recent date in the db
max_date_in_table = c.fetchone()[0]

#call the HTL api from  max_date_in_table since today
start_date = pd.to_datetime(max_date_in_table).date()
end_date = date.today()

htl = htl_api.get_historical_tonnage_list(
        port,
        vessel_class,
        laycanEndInDays,
        start_date,
        end_date,
        time=time(hour=6)
    )
htl_df = htl.to_data_frame()

#insert the new rows into the db
htl_df.to_sql('aframaxes',engine,index=True,if_exists="append")